In [1]:
import torch
from transformers import *
tokenizer = AutoTokenizer.from_pretrained("TODBERT/TOD-BERT-JNT-V1")
tod_bert = AutoModel.from_pretrained("TODBERT/TOD-BERT-JNT-V1")

In [1]:
!pip install sentencepiece

In [2]:
import pandas as pd
import numpy as np
# from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [4]:
# datapath = f'bbc-text.csv'
datapath = f'origin.csv'
df = pd.read_csv(datapath)
df.head()

,context,utterance
0,sentimental,I remember going to see the fireworks with my ...
1,sentimental,Was this a friend you were in love with， or ju...
2,sentimental,This was a best friend. I miss her.
3,sentimental,Where has she gone?
4,sentimental,We no longer talk.


In [ ]:
df.groupby(['context']).size().plot.bar()

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'afraid':0,
          'angry':1,
          'anxious':2,
          'disgusted':3,
          'embarrassed':4,
          'excited':5,
          'guilty':6,
          'jealous':7,
          'joyful':8,
          'lonely':9,
          'proud':10,
          'sad':11,
          'sentimental':12,
          'terrified':13,
          'trusting':14,
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [torch.tensor(labels[label], dtype=torch.long) for label in df['context']]
        self.texts  = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['conv']]
        # self.story = torch.Tensor(tokenizer.convert_tokens_to_ids(self.texts)).long()

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
class TODClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(TODClassifier, self).__init__()

        self.bert = tod_bert
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 15)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    #GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [65]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1780 222 223


In [66]:
EPOCHS = 5
model = TODClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 890/890 [00:53<00:00, 16.72it/s]


Epochs: 1 | Train Loss:  0.766 | Train Accuracy:  0.343 | Val Loss:  0.623 | Val Accuracy:  0.680


100%|██████████| 890/890 [00:53<00:00, 16.61it/s]


Epochs: 2 | Train Loss:  0.489 | Train Accuracy:  0.757 | Val Loss:  0.394 | Val Accuracy:  0.757


100%|██████████| 890/890 [00:53<00:00, 16.65it/s]


Epochs: 3 | Train Loss:  0.329 | Train Accuracy:  0.799 | Val Loss:  0.287 | Val Accuracy:  0.811


100%|██████████| 890/890 [01:40<00:00,  8.87it/s]


Epochs: 4 | Train Loss:  0.238 | Train Accuracy:  0.885 | Val Loss:  0.196 | Val Accuracy:  0.955


100%|██████████| 890/890 [01:45<00:00,  8.43it/s]


Epochs: 5 | Train Loss:  0.125 | Train Accuracy:  0.985 | Val Loss:  0.101 | Val Accuracy:  0.977


In [67]:
evaluate(model, df_test)

Test Accuracy:  0.996


In [ ]:
# Encode text 
input_text = "[CLS] [SYS] Hello, what can I help with you today? [USR] Find me a cheap restaurant nearby the north town."
input_tokens = tokenizer.tokenize(input_text)
story = torch.Tensor(tokenizer.convert_tokens_to_ids(input_tokens)).long()

if len(story.size()) == 1: 
    story = story.unsqueeze(0) # batch size dimension

if torch.cuda.is_available(): 
    tod_bert = tod_bert.cuda()
    story = story.cuda()

with torch.no_grad():
    input_context = {"input_ids": story, "attention_mask": (story > 0).long()}
    hiddens = tod_bert(**input_context)[0] 